# 🏦 Experimentación Controlada con German Credit Dataset

## 📋 Objetivos del Notebook

Este notebook implementa una **experimentación controlada** enfocada en:

1. **German Credit Dataset** específicamente
2. **Optimización con Optuna** para cada modelo
3. **Métricas detalladas**: AUC, PSI, Traffic Light
4. **Comparación entre**: Train, Test, Holdout
5. **Análisis por algoritmo** individual

## 🎯 Métricas de Evaluación

### Métricas Principales:
- **AUC-ROC**: Capacidad discriminante del modelo
- **PSI (Population Stability Index)**: Estabilidad de distribución entre muestras
- **Traffic Light**: Precisión en grupos de riesgo para rating bancario

### Traffic Light Methodology:
- **Verde**: Modelo predice correctamente la probabilidad de default
- **Amarillo**: Subestimación o sobrestimación leve
- **Rojo**: Subestimación o sobrestimación significativa

## 🚀 Modelos a Optimizar

1. **XGBoost** - Gradient Boosting optimizado
2. **CatBoost** - Gradient Boosting con manejo de categóricas
3. **LightGBM** - Gradient Boosting eficiente
4. **RandomForest** - Ensemble de árboles
5. **LogisticRegression** - Modelo lineal baseline

## 📊 Estructura de Evaluación

Para cada modelo optimizado:
- **Train Performance**: Métricas en datos de entrenamiento
- **Test Performance**: Métricas en datos de prueba
- **Holdout Performance**: Métricas en datos de validación
- **Comparación**: Análisis de estabilidad y generalización

---

**¡Empecemos con la experimentación controlada!** 🎯


In [1]:
# Importación de librerías
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import yaml
import logging
from tqdm import tqdm
import time
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

# Scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Modelos específicos
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# UCI Repository
from ucimlrepo import fetch_ucirepo

# Configuración
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ Librerías importadas correctamente")


C:\Users\carlo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Librerías importadas correctamente


In [ ]:
# Configuración del proyecto
PROJECT_ROOT = Path('..')
DATA_DIR = PROJECT_ROOT / 'data'
RESULTS_DIR = PROJECT_ROOT / 'results'
CONFIGS_DIR = PROJECT_ROOT / 'configs'

# Crear directorios si no existen
DATA_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(exist_ok=True)

# Configuración de experimento
RANDOM_STATE = 42
N_TRIALS = 100  # Aumentado para grillas más finas
CV_FOLDS = 5   # Folds para cross-validation

print(f"📁 Directorio del proyecto: {PROJECT_ROOT.absolute()}")
print(f"📊 Número de trials Optuna: {N_TRIALS}")
print(f"🔄 Folds de CV: {CV_FOLDS}")


📁 Directorio del proyecto: c:\Users\carlo\OneDrive\Documentos\repos\tb-grado-repo\notebooks\..
📊 Número de trials Optuna: 50
🔄 Folds de CV: 5


In [3]:
# Cargar German Credit Dataset
print("📥 Cargando German Credit Dataset...")

try:
    # Cargar dataset desde UCI Repository
    german_credit = fetch_ucirepo(id=144)
    
    # Obtener datos
    X = german_credit.data.features
    y = german_credit.data.targets
    
    print(f"✅ Dataset cargado exitosamente")
    print(f"   📊 Forma de X: {X.shape}")
    print(f"   📊 Forma de y: {y.shape}")
    print(f"   🎯 Variable objetivo: {y.columns[0]}")
    
    # Mostrar información del dataset
    print(f"\n📋 Información del dataset:")
    print(f"   Features: {list(X.columns)}")
    print(f"   Tipos de datos: {X.dtypes.value_counts().to_dict()}")
    print(f"   Valores únicos en target: {y.iloc[:, 0].value_counts().to_dict()}")
    
except Exception as e:
    print(f"❌ Error cargando dataset: {e}")
    print("🔄 Intentando cargar desde archivo local...")
    
    # Intentar cargar desde archivo local si existe
    local_file = DATA_DIR / 'german_credit.csv'
    if local_file.exists():
        df = pd.read_csv(local_file)
        X = df.drop('target', axis=1)
        y = df[['target']]
        print(f"✅ Dataset cargado desde archivo local")
    else:
        print(f"❌ No se pudo cargar el dataset")
        raise


📥 Cargando German Credit Dataset...
✅ Dataset cargado exitosamente
   📊 Forma de X: (1000, 20)
   📊 Forma de y: (1000, 1)
   🎯 Variable objetivo: class

📋 Información del dataset:
   Features: ['Attribute1', 'Attribute2', 'Attribute3', 'Attribute4', 'Attribute5', 'Attribute6', 'Attribute7', 'Attribute8', 'Attribute9', 'Attribute10', 'Attribute11', 'Attribute12', 'Attribute13', 'Attribute14', 'Attribute15', 'Attribute16', 'Attribute17', 'Attribute18', 'Attribute19', 'Attribute20']
   Tipos de datos: {dtype('O'): 13, dtype('int64'): 7}
   Valores únicos en target: {1: 700, 2: 300}


In [4]:
# Preprocesamiento de datos
print("🔧 Preprocesando datos...")

# Convertir target a binario (1 = bad credit, 0 = good credit)
y_binary = (y.iloc[:, 0] == 2).astype(int)  # 2 = bad credit en German dataset

print(f"📊 Distribución del target:")
print(f"   Good Credit (0): {(y_binary == 0).sum()} ({(y_binary == 0).mean()*100:.1f}%)")
print(f"   Bad Credit (1): {(y_binary == 1).sum()} ({(y_binary == 1).mean()*100:.1f}%)")

# Identificar variables categóricas y numéricas
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\n📋 Tipos de variables:")
print(f"   Categóricas: {len(categorical_cols)} - {categorical_cols}")
print(f"   Numéricas: {len(numerical_cols)} - {numerical_cols}")

# Codificar variables categóricas
X_encoded = X.copy()
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

print(f"✅ Variables categóricas codificadas")
print(f"📊 Forma final: {X_encoded.shape}")


🔧 Preprocesando datos...
📊 Distribución del target:
   Good Credit (0): 700 (70.0%)
   Bad Credit (1): 300 (30.0%)

📋 Tipos de variables:
   Categóricas: 13 - ['Attribute1', 'Attribute3', 'Attribute4', 'Attribute6', 'Attribute7', 'Attribute9', 'Attribute10', 'Attribute12', 'Attribute14', 'Attribute15', 'Attribute17', 'Attribute19', 'Attribute20']
   Numéricas: 7 - ['Attribute2', 'Attribute5', 'Attribute8', 'Attribute11', 'Attribute13', 'Attribute16', 'Attribute18']
✅ Variables categóricas codificadas
📊 Forma final: (1000, 20)


In [5]:
# División de datos: Train (60%) / Test (20%) / Holdout (20%)
print("📊 Dividiendo datos en Train/Test/Holdout...")

# Primera división: Train+Test (80%) / Holdout (20%)
X_temp, X_holdout, y_temp, y_holdout = train_test_split(
    X_encoded, y_binary, 
    test_size=0.2, 
    random_state=RANDOM_STATE, 
    stratify=y_binary
)

# Segunda división: Train (60%) / Test (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X_temp, y_temp, 
    test_size=0.25,  # 0.25 de 0.8 = 0.2 del total
    random_state=RANDOM_STATE, 
    stratify=y_temp
)

print(f"✅ División de datos completada:")
print(f"   🏋️ Train: {X_train.shape[0]} muestras (60%)")
print(f"   🧪 Test: {X_test.shape[0]} muestras (20%)")
print(f"   🔒 Holdout: {X_holdout.shape[0]} muestras (20%)")

# Verificar distribución del target en cada conjunto
print(f"\n📊 Distribución del target por conjunto:")
for name, y_set in [('Train', y_train), ('Test', y_test), ('Holdout', y_holdout)]:
    bad_rate = y_set.mean()
    print(f"   {name}: {bad_rate:.3f} ({y_set.sum()}/{len(y_set)})")


📊 Dividiendo datos en Train/Test/Holdout...
✅ División de datos completada:
   🏋️ Train: 600 muestras (60%)
   🧪 Test: 200 muestras (20%)
   🔒 Holdout: 200 muestras (20%)

📊 Distribución del target por conjunto:
   Train: 0.300 (180/600)
   Test: 0.300 (60/200)
   Holdout: 0.300 (60/200)


In [ ]:
# Clase para métricas de evaluación
class CreditScoringMetrics:
    """
    Clase para calcular métricas específicas de scoring crediticio
    """
    
    @staticmethod
    def calculate_auc_roc(y_true, y_pred_proba):
        """
        Calcula AUC-ROC
        """
        return roc_auc_score(y_true, y_pred_proba)
    
    @staticmethod
    def calculate_psi(expected, actual, bins=10):
        """
        Calcula Population Stability Index (PSI)
        
        Args:
            expected: Distribución esperada (train)
            actual: Distribución actual (test/holdout)
            bins: Número de bins para discretizar
        
        Returns:
            PSI value
        """
        # Crear bins basados en la distribución esperada
        breakpoints = np.linspace(0, 1, bins + 1)
        breakpoints[0] = -np.inf
        breakpoints[-1] = np.inf
        
        # Discretizar ambas distribuciones
        expected_binned = pd.cut(expected, bins=breakpoints, labels=False)
        actual_binned = pd.cut(actual, bins=breakpoints, labels=False)
        
        # Calcular frecuencias
        expected_freq = pd.Series(expected_binned).value_counts(normalize=True, sort=False)
        actual_freq = pd.Series(actual_binned).value_counts(normalize=True, sort=False)
        
        # Asegurar que ambos tengan los mismos bins
        for i in range(bins):
            if i not in expected_freq.index:
                expected_freq[i] = 0
            if i not in actual_freq.index:
                actual_freq[i] = 0
        
        expected_freq = expected_freq.sort_index()
        actual_freq = actual_freq.sort_index()
        
        # Calcular PSI
        psi = 0
        for i in range(bins):
            if expected_freq.iloc[i] > 0:
                psi += (actual_freq.iloc[i] - expected_freq.iloc[i]) * \
                       np.log(actual_freq.iloc[i] / expected_freq.iloc[i])
        
        return psi
    
    @staticmethod
    def calculate_traffic_light(y_true, y_pred_proba, n_groups=10):
        """
        Calcula Traffic Light para grupos de riesgo por deciles
        
        Args:
            y_true: Valores reales
            y_pred_proba: Probabilidades predichas
            n_groups: Número de grupos de riesgo (deciles)
        
        Returns:
            Dict con estadísticas de Traffic Light
        """
        # Crear DataFrame con datos
        df = pd.DataFrame({
            'actual': y_true,
            'predicted': y_pred_proba
        })
        
        # Crear deciles basados en probabilidades predichas (descendente)
        # Decil 1 = mayor riesgo, Decil 10 = menor riesgo
        df['decile'] = pd.qcut(df['predicted'], q=n_groups, labels=False, duplicates='drop') + 1
        
        # Calcular métricas por decil
        group_stats = []
        for decile in range(1, n_groups + 1):
            decile_data = df[df['decile'] == decile]
            if len(decile_data) > 0:
                actual_rate = decile_data['actual'].mean()
                predicted_rate = decile_data['predicted'].mean()
                
                # Determinar color del semáforo
                diff = abs(actual_rate - predicted_rate)
                if diff <= 0.05:  # 5% de tolerancia
                    color = 'green'
                elif diff <= 0.10:  # 10% de tolerancia
                    color = 'yellow'
                else:
                    color = 'red'
                
                group_stats.append({
                    'decile': decile,
                    'actual_rate': actual_rate,
                    'predicted_rate': predicted_rate,
                    'difference': diff,
                    'color': color,
                    'size': len(decile_data),
                    'min_prob': decile_data['predicted'].min(),
                    'max_prob': decile_data['predicted'].max(),
                    'avg_prob': decile_data['predicted'].mean()
                })
        
        # Calcular estadísticas generales
        colors = [stat['color'] for stat in group_stats]
        green_pct = colors.count('green') / len(colors) * 100
        yellow_pct = colors.count('yellow') / len(colors) * 100
        red_pct = colors.count('red') / len(colors) * 100
        
        return {
            'group_stats': group_stats,
            'green_percentage': green_pct,
            'yellow_percentage': yellow_pct,
            'red_percentage': red_pct,
            'total_groups': len(group_stats)
        }
    
    @classmethod
    def evaluate_model(cls, y_true, y_pred_proba, y_train_proba=None):
        """
        Evalúa un modelo con todas las métricas
        
        Args:
            y_true: Valores reales
            y_pred_proba: Probabilidades predichas
            y_train_proba: Probabilidades en train (para PSI)
        
        Returns:
            Dict con todas las métricas
        """
        results = {}
        
        # AUC-ROC
        results['auc_roc'] = cls.calculate_auc_roc(y_true, y_pred_proba)
        
        # PSI (si se proporcionan datos de train)
        if y_train_proba is not None:
            results['psi'] = cls.calculate_psi(y_train_proba, y_pred_proba)
        
        # Traffic Light
        traffic_light = cls.calculate_traffic_light(y_true, y_pred_proba)
        results['traffic_light'] = traffic_light
        
        return results

print("✅ Clase de métricas creada")


✅ Clase de métricas creada


In [32]:
# Clase para optimización con Optuna
class OptunaOptimizer:
    """
    Clase para optimizar modelos con Optuna
    """
    
    def __init__(self, X_train, y_train, cv_folds=5, n_trials=50):
        self.X_train = X_train
        self.y_train = y_train
        self.cv_folds = cv_folds
        self.n_trials = n_trials
        self.best_params = {}
        self.best_scores = {}
        
    def optimize_xgboost(self):
        """
        Optimiza XGBoost con Optuna - Enfocado en reducir overfitting
        """
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 150),  # Reducido
                'max_depth': trial.suggest_int('max_depth', 2, 8),  # Más restrictivo
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.10),  # Más conservador
                'subsample': trial.suggest_float('subsample', 0.1, 0.5),  # Más regularización
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 0.9),  # Más regularización
                'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 125.0),  # L1 regularización
                'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 125.0),  # L2 regularización
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),  # Control de overfitting
                'gamma': trial.suggest_float('gamma', 0, 2),  # Regularización adicional
                'random_state': RANDOM_STATE
            }
            
            model = xgb.XGBClassifier(**params)
            cv_scores = cross_val_score(model, self.X_train, self.y_train, 
                                      cv=self.cv_folds, scoring='roc_auc')
            return cv_scores.mean()
        
        study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=RANDOM_STATE))
        study.optimize(objective, n_trials=self.n_trials)
        
        self.best_params['xgboost'] = study.best_params
        self.best_scores['xgboost'] = study.best_value
        
        return study.best_params
    
    def optimize_lightgbm(self):
        """
        Optimiza LightGBM con Optuna - Enfocado en reducir overfitting
        """
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 200),  # Reducido
                'max_depth': trial.suggest_int('max_depth', 2, 10),  # Más restrictivo
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15),  # Más conservador
                'subsample': trial.suggest_float('subsample', 0.1, 0.5),  # Más regularización
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 0.9),  # Más regularización
                'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 125.0),  # L1 regularización
                'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 125.0),  # L2 regularización
                'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),  # Control de overfitting
                'min_split_gain': trial.suggest_float('min_split_gain', 0, 1),  # Regularización adicional
                'random_state': RANDOM_STATE,
                'verbose': -1
            }
            
            model = lgb.LGBMClassifier(**params)
            cv_scores = cross_val_score(model, self.X_train, self.y_train, 
                                      cv=self.cv_folds, scoring='roc_auc')
            return cv_scores.mean()
        
        study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=RANDOM_STATE))
        study.optimize(objective, n_trials=self.n_trials)
        
        self.best_params['lightgbm'] = study.best_params
        self.best_scores['lightgbm'] = study.best_value
        
        return study.best_params
    
    def optimize_catboost(self):
        """
        Optimiza CatBoost con Optuna - Enfocado en reducir overfitting
        """
        def objective(trial):
            params = {
                'iterations': trial.suggest_int('iterations', 50, 250),  # Reducido
                'depth': trial.suggest_int('depth', 2, 6),  # Más restrictivo
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),  # Más conservador
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 100),  # L2 regularización
                'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli']),
                'subsample': trial.suggest_float('subsample', 0.7, 0.9),  # Más regularización
                'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 0.5),  # Regularización
                'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 50),  # Control de overfitting
                'random_seed': RANDOM_STATE,
                'verbose': False
            }
            
            model = cb.CatBoostClassifier(**params)
            cv_scores = cross_val_score(model, self.X_train, self.y_train, 
                                      cv=self.cv_folds, scoring='roc_auc')
            return cv_scores.mean()
        
        study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=RANDOM_STATE))
        study.optimize(objective, n_trials=self.n_trials)
        
        self.best_params['catboost'] = study.best_params
        self.best_scores['catboost'] = study.best_value
        
        return study.best_params
    
    def optimize_random_forest(self):
        """
        Optimiza Random Forest con Optuna - Enfocado en reducir overfitting
        """
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 200),  # Reducido
                'max_depth': trial.suggest_int('max_depth', 3, 10),  # Más restrictivo
                'min_samples_split': trial.suggest_int('min_samples_split', 5, 20),  # Más restrictivo
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 10),  # Más restrictivo
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),  # Sin None
                'bootstrap': True,  # Bootstrap para regularización
                'max_samples': trial.suggest_float('max_samples', 0.7, 0.9),  # Submuestreo
                'random_state': RANDOM_STATE
            }
            
            model = RandomForestClassifier(**params)
            cv_scores = cross_val_score(model, self.X_train, self.y_train, 
                                      cv=self.cv_folds, scoring='roc_auc')
            return cv_scores.mean()
        
        study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=RANDOM_STATE))
        study.optimize(objective, n_trials=self.n_trials)
        
        self.best_params['random_forest'] = study.best_params
        self.best_scores['random_forest'] = study.best_value
        
        return study.best_params
    
    def optimize_logistic_regression(self):
        """
        Optimiza Logistic Regression con Optuna - Enfocado en reducir overfitting
        """
        def objective(trial):
            params = {
                'C': trial.suggest_float('C', 0.01, 10, log=True),  # Más restrictivo
                'penalty': trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet']),
                'solver': 'saga',  # Compatible con elasticnet
                'l1_ratio': trial.suggest_float('l1_ratio', 0.1, 0.9),  # Para elasticnet
                'max_iter': 1000,  # Más iteraciones
                'random_state': RANDOM_STATE
            }
            
            model = LogisticRegression(**params)
            cv_scores = cross_val_score(model, self.X_train, self.y_train, 
                                      cv=self.cv_folds, scoring='roc_auc')
            return cv_scores.mean()
        
        study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=RANDOM_STATE))
        study.optimize(objective, n_trials=self.n_trials)
        
        self.best_params['logistic_regression'] = study.best_params
        self.best_scores['logistic_regression'] = study.best_value
        
        return study.best_params

print("✅ Clase de optimización creada")


✅ Clase de optimización creada


In [33]:
# Inicializar optimizador
optimizer = OptunaOptimizer(X_train, y_train, cv_folds=CV_FOLDS, n_trials=N_TRIALS)

print(f"🚀 Optimizador inicializado")
print(f"   📊 Datos de entrenamiento: {X_train.shape}")
print(f"   🔄 Folds de CV: {CV_FOLDS}")
print(f"   🎯 Trials por modelo: {N_TRIALS}")


🚀 Optimizador inicializado
   📊 Datos de entrenamiento: (600, 20)
   🔄 Folds de CV: 5
   🎯 Trials por modelo: 50


In [34]:
# Optimizar todos los modelos
print("🔥 OPTIMIZANDO TODOS LOS MODELOS...")
print("="*50)

# Lista de modelos a optimizar
models_to_optimize = [
    ('XGBoost', optimizer.optimize_xgboost),
    ('LightGBM', optimizer.optimize_lightgbm),
    ('CatBoost', optimizer.optimize_catboost),
    ('RandomForest', optimizer.optimize_random_forest),
    ('LogisticRegression', optimizer.optimize_logistic_regression)
]

# Optimizar cada modelo
for model_name, optimize_func in models_to_optimize:
    print(f"\n🔥 Optimizando {model_name}...")
    start_time = time.time()
    
    try:
        best_params = optimize_func()
        end_time = time.time()
        
        print(f"✅ {model_name} optimizado en {end_time - start_time:.1f} segundos")
        print(f"   🏆 Mejor CV Score: {optimizer.best_scores[model_name.lower().replace(' ', '_')]:.4f}")
        print(f"   ⚙️ Mejores parámetros: {best_params}")
        
    except Exception as e:
        print(f"❌ Error optimizando {model_name}: {e}")

print(f"\n✅ Optimización completada para todos los modelos")


[I 2025-10-17 15:39:17,272] A new study created in memory with name: no-name-81b4ed15-fdd7-43c0-a2c2-716c52421b09


🔥 OPTIMIZANDO TODOS LOS MODELOS...

🔥 Optimizando XGBoost...


[I 2025-10-17 15:39:17,871] Trial 0 finished with value: 0.5 and parameters: {'n_estimators': 87, 'max_depth': 8, 'learning_rate': 0.07587945476302646, 'subsample': 0.3394633936788146, 'colsample_bytree': 0.7312037280884873, 'reg_alpha': 19.583715589991712, 'reg_lambda': 7.354643159808113, 'min_child_weight': 9, 'gamma': 1.2022300234864176}. Best is trial 0 with value: 0.5.
[I 2025-10-17 15:39:18,834] Trial 1 finished with value: 0.5 and parameters: {'n_estimators': 121, 'max_depth': 2, 'learning_rate': 0.0972918866945795, 'subsample': 0.4329770563201687, 'colsample_bytree': 0.7424678221356552, 'reg_alpha': 22.80993840416687, 'reg_lambda': 23.007223280693886, 'min_child_weight': 4, 'gamma': 1.0495128632644757}. Best is trial 0 with value: 0.5.
[I 2025-10-17 15:39:19,405] Trial 2 finished with value: 0.5 and parameters: {'n_estimators': 93, 'max_depth': 4, 'learning_rate': 0.06506676052501416, 'subsample': 0.15579754426081674, 'colsample_bytree': 0.7584289297070436, 'reg_alpha': 45.8585

✅ XGBoost optimizado en 31.9 segundos
   🏆 Mejor CV Score: 0.7715
   ⚙️ Mejores parámetros: {'n_estimators': 149, 'max_depth': 7, 'learning_rate': 0.04397484995849731, 'subsample': 0.49840679574448554, 'colsample_bytree': 0.7979409097155367, 'reg_alpha': 2.7567857408156784, 'reg_lambda': 75.31385949233461, 'min_child_weight': 6, 'gamma': 0.8014695857116751}

🔥 Optimizando LightGBM...


[I 2025-10-17 15:39:49,513] Trial 1 finished with value: 0.6892857142857143 and parameters: {'n_estimators': 156, 'max_depth': 2, 'learning_rate': 0.1457873793026792, 'subsample': 0.4329770563201687, 'colsample_bytree': 0.7424678221356552, 'reg_alpha': 22.80993840416687, 'reg_lambda': 23.007223280693886, 'min_child_samples': 22, 'min_split_gain': 0.5247564316322378}. Best is trial 0 with value: 0.7014550264550264.
[I 2025-10-17 15:39:49,648] Trial 2 finished with value: 0.5 and parameters: {'n_estimators': 115, 'max_depth': 4, 'learning_rate': 0.09565940526113312, 'subsample': 0.15579754426081674, 'colsample_bytree': 0.7584289297070436, 'reg_alpha': 45.8585942273821, 'reg_lambda': 57.06314102870779, 'min_child_samples': 42, 'min_split_gain': 0.19967378215835974}. Best is trial 0 with value: 0.7014550264550264.
[I 2025-10-17 15:39:49,845] Trial 3 finished with value: 0.7518849206349207 and parameters: {'n_estimators': 127, 'max_depth': 7, 'learning_rate': 0.01650305778079968, 'subsample

✅ LightGBM optimizado en 12.8 segundos
   🏆 Mejor CV Score: 0.7912
   ⚙️ Mejores parámetros: {'n_estimators': 110, 'max_depth': 10, 'learning_rate': 0.07409265653208359, 'subsample': 0.35376581331497314, 'colsample_bytree': 0.898273859191043, 'reg_alpha': 0.19489892822749993, 'reg_lambda': 0.9938694645855488, 'min_child_samples': 38, 'min_split_gain': 0.5753595623166196}

🔥 Optimizando CatBoost...
❌ Error optimizando CatBoost: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\carlo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.

[I 2025-10-17 15:40:03,846] Trial 0 finished with value: 0.8146164021164021 and parameters: {'n_estimators': 106, 'max_depth': 10, 'min_samples_split': 16, 'min_samples_leaf': 7, 'max_features': 'sqrt', 'max_samples': 0.7116167224336398}. Best is trial 0 with value: 0.8146164021164021.
[I 2025-10-17 15:40:06,770] Trial 1 finished with value: 0.8095899470899471 and parameters: {'n_estimators': 180, 'max_depth': 7, 'min_samples_split': 16, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_samples': 0.7424678221356552}. Best is trial 0 with value: 0.8146164021164021.
[I 2025-10-17 15:40:08,625] Trial 2 finished with value: 0.8070767195767197 and parameters: {'n_estimators': 77, 'max_depth': 4, 'min_samples_split': 9, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'max_samples': 0.8223705789444758}. Best is trial 0 with value: 0.8146164021164021.
[I 2025-10-17 15:40:10,082] Trial 3 finished with value: 0.8066137566137564 and parameters: {'n_estimators': 71, 'max_depth': 5, 'min_samples_s

✅ RandomForest optimizado en 106.8 segundos
❌ Error optimizando RandomForest: 'randomforest'

🔥 Optimizando LogisticRegression...


[I 2025-10-17 15:41:50,436] Trial 0 finished with value: 0.5261243386243386 and parameters: {'C': 0.13292918943162169, 'penalty': 'l1', 'l1_ratio': 0.22481491235394924}. Best is trial 0 with value: 0.5261243386243386.
[I 2025-10-17 15:41:51,739] Trial 1 finished with value: 0.5263227513227513 and parameters: {'C': 0.029375384576328288, 'penalty': 'l2', 'l1_ratio': 0.6664580622368363}. Best is trial 1 with value: 0.5263227513227513.
[I 2025-10-17 15:41:53,290] Trial 2 finished with value: 0.5188492063492063 and parameters: {'C': 0.011527987128232402, 'penalty': 'l1', 'l1_ratio': 0.24545997376568052}. Best is trial 1 with value: 0.5263227513227513.
[I 2025-10-17 15:41:54,603] Trial 3 finished with value: 0.5263227513227513 and parameters: {'C': 0.03549878832196503, 'penalty': 'l2', 'l1_ratio': 0.3329833121584336}. Best is trial 1 with value: 0.5263227513227513.
[I 2025-10-17 15:41:56,256] Trial 4 finished with value: 0.5263227513227513 and parameters: {'C': 0.6847920095574779, 'penalty':

✅ LogisticRegression optimizado en 80.6 segundos
❌ Error optimizando LogisticRegression: 'logisticregression'

✅ Optimización completada para todos los modelos


In [35]:
# Resumen de optimización
print("📊 RESUMEN DE OPTIMIZACIÓN")
print("="*50)

for model_name, score in optimizer.best_scores.items():
    print(f"{model_name.upper()}: {score:.4f}")

# Encontrar el mejor modelo
best_model_name = max(optimizer.best_scores, key=optimizer.best_scores.get)
best_score = optimizer.best_scores[best_model_name]

print(f"\n🏆 MEJOR MODELO: {best_model_name.upper()}")
print(f"📊 Mejor CV Score: {best_score:.4f}")


📊 RESUMEN DE OPTIMIZACIÓN
XGBOOST: 0.7715
LIGHTGBM: 0.7912
RANDOM_FOREST: 0.8196
LOGISTIC_REGRESSION: 0.5265

🏆 MEJOR MODELO: RANDOM_FOREST
📊 Mejor CV Score: 0.8196


In [37]:
# Entrenar modelos optimizados y evaluar
print("🏋️ Entrenando modelos optimizados...")

# Crear modelos con mejores parámetros
models = {
    'XGBoost': xgb.XGBClassifier(**optimizer.best_params['xgboost'], random_state=RANDOM_STATE),
    'LightGBM': lgb.LGBMClassifier(**optimizer.best_params['lightgbm'], random_state=RANDOM_STATE, verbose=-1),
   # 'CatBoost': cb.CatBoostClassifier(**optimizer.best_params['catboost'], random_seed=RANDOM_STATE, verbose=False),
    'RandomForest': RandomForestClassifier(**optimizer.best_params['random_forest'], random_state=RANDOM_STATE),
   # 'LogisticRegression': LogisticRegression(**optimizer.best_params['logistic_regression'], random_state=RANDOM_STATE)
}

# Entrenar todos los modelos
trained_models = {}
for name, model in models.items():
    print(f"   Entrenando {name}...")
    model.fit(X_train, y_train)
    trained_models[name] = model

print("✅ Todos los modelos entrenados")


🏋️ Entrenando modelos optimizados...
   Entrenando XGBoost...
   Entrenando LightGBM...
   Entrenando RandomForest...
✅ Todos los modelos entrenados


In [38]:
# Evaluar modelos en Train, Test y Holdout
print("📊 EVALUANDO MODELOS EN TRAIN/TEST/HOLDOUT...")
print("="*60)

results = {}

for model_name, model in trained_models.items():
    print(f"\n🔍 Evaluando {model_name}...")
    
    # Predicciones en cada conjunto
    train_proba = model.predict_proba(X_train)[:, 1]
    test_proba = model.predict_proba(X_test)[:, 1]
    holdout_proba = model.predict_proba(X_holdout)[:, 1]
    
    # Evaluar en cada conjunto
    train_metrics = CreditScoringMetrics.evaluate_model(y_train, train_proba)
    test_metrics = CreditScoringMetrics.evaluate_model(y_test, test_proba, train_proba)
    holdout_metrics = CreditScoringMetrics.evaluate_model(y_holdout, holdout_proba, train_proba)
    
    results[model_name] = {
        'train': train_metrics,
        'test': test_metrics,
        'holdout': holdout_metrics
    }
    
    # Mostrar resultados
    print(f"   📈 Train  - AUC: {train_metrics['auc_roc']:.4f}, PSI: N/A, Green: {train_metrics['traffic_light']['green_percentage']:.1f}%")
    print(f"   🧪 Test   - AUC: {test_metrics['auc_roc']:.4f}, PSI: {test_metrics['psi']:.4f}, Green: {test_metrics['traffic_light']['green_percentage']:.1f}%")
    print(f"   🔒 Holdout - AUC: {holdout_metrics['auc_roc']:.4f}, PSI: {holdout_metrics['psi']:.4f}, Green: {holdout_metrics['traffic_light']['green_percentage']:.1f}%")


📊 EVALUANDO MODELOS EN TRAIN/TEST/HOLDOUT...

🔍 Evaluando XGBoost...
   📈 Train  - AUC: 0.8091, PSI: N/A, Green: 30.0%
   🧪 Test   - AUC: 0.7139, PSI: 0.0557, Green: 30.0%
   🔒 Holdout - AUC: 0.7779, PSI: 0.0083, Green: 10.0%

🔍 Evaluando LightGBM...
   📈 Train  - AUC: 0.9353, PSI: N/A, Green: 10.0%
   🧪 Test   - AUC: 0.7455, PSI: inf, Green: 40.0%
   🔒 Holdout - AUC: 0.7913, PSI: inf, Green: 40.0%

🔍 Evaluando RandomForest...
   📈 Train  - AUC: 0.9525, PSI: N/A, Green: 0.0%
   🧪 Test   - AUC: 0.7469, PSI: 0.2022, Green: 40.0%
   🔒 Holdout - AUC: 0.7963, PSI: 0.0751, Green: 40.0%


In [46]:
exec(open('simple_exec.py').read())

SyntaxError: source code string cannot contain null bytes (<string>)

In [47]:
exec(open('complete_traffic_light.py').read())

SyntaxError: source code string cannot contain null bytes (<string>)

In [48]:
exec(open('traffic_light_fix.py').read())

# Evaluate models with statistical Traffic Light
print('Evaluating models with statistical Traffic Light...')
results_statistical = evaluate_models_statistical(
    trained_models, X_train, y_train, X_test, y_test, X_holdout, y_holdout
)

print('Statistical evaluation completed')

Statistical Traffic Light functions loaded successfully
Evaluating models with statistical Traffic Light...
Evaluating XGBoost with statistical Traffic Light...


AttributeError: module 'scipy.stats' has no attribute 'binom_test'

In [49]:
exec(open('simple_exec.py').read())

SyntaxError: source code string cannot contain null bytes (<string>)

In [52]:
# Traffic Light Statistical Analysis
exec(open('traffic_light_fix.py').read())

print('Evaluating models with statistical Traffic Light...')
results_statistical = evaluate_models_statistical(
    trained_models, X_train, y_train, X_test, y_test, X_holdout, y_holdout
)

print('Statistical evaluation completed')

Statistical Traffic Light functions loaded successfully
Evaluating models with statistical Traffic Light...
Evaluating XGBoost with statistical Traffic Light...


AttributeError: module 'scipy.stats' has no attribute 'binom_test'

In [53]:
# Traffic Light Statistical Analysis
exec(open('traffic_light_fix.py').read())

print('Evaluating models with statistical Traffic Light...')
results_statistical = evaluate_models_statistical(
    trained_models, X_train, y_train, X_test, y_test, X_holdout, y_holdout
)

print('Statistical evaluation completed')

Statistical Traffic Light functions loaded successfully
Evaluating models with statistical Traffic Light...
Evaluating XGBoost with statistical Traffic Light...
   Train  - Green: 50.0%, Yellow: 0.0%, Red: 50.0%
   Test   - Green: 90.0%, Yellow: 10.0%, Red: 0.0%
   Holdout - Green: 90.0%, Yellow: 0.0%, Red: 10.0%
Evaluating LightGBM with statistical Traffic Light...
   Train  - Green: 60.0%, Yellow: 10.0%, Red: 30.0%
   Test   - Green: 100.0%, Yellow: 0.0%, Red: 0.0%
   Holdout - Green: 100.0%, Yellow: 0.0%, Red: 0.0%
Evaluating RandomForest with statistical Traffic Light...
   Train  - Green: 50.0%, Yellow: 0.0%, Red: 50.0%
   Test   - Green: 100.0%, Yellow: 0.0%, Red: 0.0%
   Holdout - Green: 100.0%, Yellow: 0.0%, Red: 0.0%
Statistical evaluation completed


In [54]:
exec(open('execute_basel_traffic_light.py').read())

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 747: character maps to <undefined>